In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [13]:
import torch
import csv

import cv2
import numpy as np

from tqdm import tqdm

import os, os.path as osp
from pathlib import Path

from MinkowskiEngine.utils import sparse_quantize

from plyfile import PlyElement, PlyData

from datasets.scannet.utils import load_ply

# File names and params

In [3]:
DATA_DIR = Path('/mnt/data/scannet/scans/')

voxel_size = 0.05

scan_id = 'scene0000_00'
scan_dir = DATA_DIR / scan_id
label_file = DATA_DIR.parent / 'scannetv2-labels.combined.tsv'

input_file = f'{scan_id}_vh_clean_2.ply' 
gt_file = f'{scan_id}_vh_clean_2.labels.ply' 

In [4]:
# read colors from input file
_, rgb, _ = load_ply(scan_dir / input_file)
# read coords and labels from GT file
coords, _, labels = load_ply(scan_dir / gt_file, read_label=True)
print(coords.shape, rgb.shape, labels.shape)

print('NYU Labels:', labels.shape, 'Range:', labels.min(), labels.max(), 
      'Unique:', len(np.unique(labels)))

(81369, 3) (81369, 3) (81369,)
NYU Labels: (81369,) Range: 0 40 Unique: 24


# Voxelize with Minkowski Engine

In [5]:
coords_vox, rgb_vox, labels_vox = sparse_quantize(coords, rgb, labels, quantization_size=voxel_size)
print(coords_vox.shape, rgb_vox.shape, labels_vox.shape)

(53518, 3) (53518, 3) (53518,)


In [28]:
print(coords_vox.dtype, rgb_vox.dtype, labels_vox.dtype)
print(labels_vox.min(), labels_vox.max())

int32 float32 int32
-100 40


In [30]:
print(rgb_vox.min(axis=0), rgb_vox.max(axis=0))

[5. 5. 1.] [255. 255. 255.]


In [29]:
(labels_vox == -100).sum()

1943

# Compare coordinates before and after voxelization

In [7]:
coords.min(axis=0), coords.max(axis=0)

(array([-0.01656916, -0.00354378, -0.00058457], dtype=float32),
 array([8.408409, 8.740397, 3.024793], dtype=float32))

In [8]:
coords_vox.min(axis=0), coords_vox.max(axis=0)

(array([-1, -1, -1], dtype=int32), array([168, 174,  60], dtype=int32))

In [9]:
coords_vox.min(axis=0) * voxel_size, coords_vox.max(axis=0) * voxel_size

(array([-0.05, -0.05, -0.05]), array([8.4, 8.7, 3. ]))

In [10]:
coords_vox[0]

array([10, 90,  5], dtype=int32)

## save point cloud to file

In [22]:
# store everything in one array for the PLY file
arr = np.array([tuple(coords_vox[i]) + tuple(rgb_vox[i]) + (labels_vox[i],) for i in range(len(coords_vox))], 
               dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4'), ('red', 'f4'), ('green', 'f4'), ('blue', 'f4'), ('label', 'i4')])

elem = PlyElement.describe(arr, 'vertex')

out_file = scan_dir / f'{scan_id}_voxelized.ply'
print(f'Saving to: {out_file}')
PlyData([elem]).write(out_file)

Saving to: /mnt/data/scannet/scans/scene0000_00/scene0000_00_voxelized.ply


# Read back from file

In [27]:
c, r, l = load_ply(out_file, read_label=True)
print(c.shape, r.shape, l.shape)
print(c.dtype, r.dtype, l.dtype)
print(c.min(axis=0), c.max(axis=0))
print(r.min(axis=0), r.max(axis=0))
print(l.min(), l.max())

(53518, 3) (53518, 3) (53518,)
float32 float32 int32
[-1. -1. -1.] [168. 174.  60.]
[5. 5. 1.] [255. 255. 255.]
-100 40
